# Pre-process corn pictures: equalize and crop
## CSS844 module 2
### Author: Miles Roberts
### Last updated: 2021-03-03
### Memo: compiled pre-write ideas into single script

In [6]:
#The following code snip-it reads any file from the internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
from skimage import exposure #histogram equalization
import colorsys #To convert to rbg to hsv color space
import matplotlib.colors as colors
import os #For getting list of files
from scipy import ndimage #For performing erosion and dilation

#Get list of photo names
fileNames = ['IMG_8277.jpg','IMG_8280.jpg','IMG_8333.jpg','IMG_8338.jpg','IMG_8342.jpg','IMG_8343.jpg']

#Define thresholds for isolating tray in photos
hmin = -0.01
hmax = 1.01
smin = 0.05
smax = 1.01
vmin = -0.01
vmax = 1.01

#Function for calculating run lengths in a binary array
#function is from: https://stackoverflow.com/questions/1066758/find-length-of-sequences-of-identical-values-in-a-numpy-array-run-length-encodi
def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

#Loop over photos and crop them
for fileName in fileNames:
    #Load picture
    im = imread(fileName)
    
    #Histogram equalization
    im2 = exposure.equalize_hist(im)
    
    #convert from rgb to hsv color space, pull out matrices
    hsv = colors.rgb_to_hsv(im2)
    h = hsv[:,:,0]; #hue matrix
    s = hsv[:,:,1]; #saturation matrix
    v = hsv[:,:,2]; #value matrix (i.e. brightness)
    
    #Convert to binary image based on thresholds
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    b_img = (b_img & 
        (s > smin) & (s < smax) & 
        (v > vmin) & (v < vmax));
    
    #Clean up binary image with erosion and dilation
    b2 = ndimage.binary_erosion(b_img, iterations = 10)
    b3 = ndimage.binary_dilation(b2, iterations = 20)
    
    #Label objects in binary image
    lab, num_features = ndimage.measurements.label(b3)
    
    #Sum togther rows and columns of binary array to determine which pixels represent the tray (labeled as object 1) 
    a1 = np.sum(lab==1,axis=1)
    a0 = np.sum(lab==1,axis=0)
    
    #Convert binary arrays to logical arrays. Now just need to find longest run of False elements in each array
    al0 = a0 < max(a0)
    al1 = a1 < max(a1)
    
    #Calculate run lengths
    runLengths0 = rle(al0)
    runLengths1 = rle(al1)
    
    #Focus on columns
    ##Find index of where longest run begins
    runs0 = runLengths0[0]
    positions0 =  runLengths0[1]
    maxRun0 = max(runs0)
    result = np.where(runs0 == maxRun0)
    ##Calculate where longest run ends
    index = np.asarray(result)
    startCol = positions0[index].tolist()[0][0] - 120
    endCol = startCol + maxRun0 + 240
    
    #Focus on rows
    ##Find index of where longest run begins
    runs1 = runLengths1[0]
    positions1 =  runLengths1[1]
    maxRun1 = max(runs1)
    result = np.where(runs1 == maxRun1)
    ##Calculate where longest run ends
    index = np.asarray(result)
    startRow = positions1[index].tolist()[0][0] - 120
    endRow = startRow + maxRun1 + 240
    
    #Crop and save image
    im3 = im2[startRow:endRow,startCol:endCol]
    imsave("cropped_modified_again_BIG_excellent" + fileName, im3)


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:
#The following code snip-it reads any file from the internet and saves it to your local directory.
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
filename = 'cropped_modified_again_BIG_excellentIMG_8277.jpg'

im = imread(filename)
im = exposure.equalize_hist(im)
plt.figure(figsize=(20,10));
plt.imshow(im)

In [ ]:
hmin = -0.01
hmax = 0.15
smin = 0.30
smax = 0.70
vmin = 0.50
vmax = 1.01


hsv = colors.rgb_to_hsv(im)
h = hsv[:,:,0];
s = hsv[:,:,1];
v = hsv[:,:,2];
    
    # trick because the color space wraps
if hmin > hmax:
    b_img = (h > hmin) | (h < hmax)
else:
    b_img = (h > hmin) & (h < hmax);
    
    
b_img = (b_img & 
    (s > smin) & (s < smax) & 
    (v > vmin) & (v < vmax));

b1 = ndimage.binary_erosion(b_img, iterations = 5)
b2 = ndimage.binary_dilation(b1, iterations = 30)
b3 = ndimage.binary_erosion(b2, iterations = 5)

plt.imshow(b3)